<a href="https://colab.research.google.com/github/titocampos/Arquivos/blob/master/Scrapping_FIFA_World_Cup_2018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scrapping FIFA World Cup 2018

Os dados que você precisará nem sempre podem estar à vista. Mas a boa notícia é que estão lá, escondidos nas páginas da web. Você só precisa rastrear essas páginas para extraí-lo. É isso que o Web Scraping é. Vamos construir um dataset usando Python e BeautifulSoup para coletar dados da Copa do Mundo da FIFA 2018.

In [0]:
import requests
from bs4 import BeautifulSoup
import pandas
from collections import OrderedDict
import time


Utilizaremos a biblioteca **requests** para fazer o download da página. Já a 
**BeautifulSoup** usaremos esta para analisar a página HTML para extrair os dados de que precisamos.

In [0]:
page = requests.get("https://fifa.com/worldcup/players/player/201200/profile.html")
soup = BeautifulSoup(page.content,"html.parser")
print(soup.prettify())

Scraping é mais fácil quando o desenvolvedor da Web faz sua parte diligentemente, porque usaremos atributos HTML para saber quais dados exatos devem ser retornados. Na celula abaixo retornanmos o nome do jogador baseado na classe do div que ele está contido. 

In [0]:
print(soup.find("div",{"class":"fi-p__name"}).text.replace("\n","").strip())

CRISTIANO RONALDO


Analisando o código fonte do site, encontrei este link onde todas as IDs de jogadores são mencionadas. Por isso, peguei esta página para listar os ID´s dos jogadores.

In [0]:
id_list = []
request = requests.get("https://www.fifa.com/worldcup/players/_libraries/byposition/[id]/_players-list")
soup = BeautifulSoup(request.content,"html.parser")
all_players = soup.find_all("a","fi-p--link")

for player in all_players:
    id_list.append(player['data-player-id'])

df = pandas.DataFrame({
"Ids":id_list
})
df.to_csv('player_ids.csv', index = False)
print(df,"\n Success")

Agora que temos os IDs e sabemos como buscar cada parâmetro, basta passar esses IDs e executar toda a iteração.

In [0]:
player_ids = pandas.read_csv("player_ids.csv")
ids = player_ids["Ids"]

base_url = "https://www.fifa.com/worldcup/players/player/"
player_list = []

for id in ids[0:30]:
    d=OrderedDict()
    request = requests.get(base_url+str(id)+"/profile.html")
    content = request.content
    soup = BeautifulSoup(content,"html.parser")
    d['id'] = id
    d['Name'] = soup.find("div",{"class":"fi-p__name"}).text.replace("\n","").strip()
    d['Country'] = soup.find("div",{"class":"fi-p__country"}).text.replace("\n","").strip()
    d['Role'] = soup.find("div",{"class":"fi-p__role"}).text.replace("\n","").strip()
    d['Age'] = soup.find("div",{"class":"fi-p__profile-number__number"}).text.replace("\n","").strip()
    d['Height(cm)'] = soup.find_all("div",{"class":"fi-p__profile-number__number"})[1].text.replace("\n","").strip()
    d['International Caps'] = soup.find_all("div",{"class":"fi-p__profile-number__number"})[2].text.replace("\n","").strip()
    d['International Goals'] = soup.find_all("div",{"class":"fi-p__profile-number__number"})[3].text.replace("\n","").strip()
    print(d['Name'])
    player_list.append(d)
    time.sleep(0.2)

df = pandas.DataFrame(player_list)
df.to_csv('Players_info.csv', index = False)
print("Success \n")

Pedro GALLESE
Alberto RODRIGUEZ
Aldo CORZO
Anderson SANTAMARIA
Miguel ARAUJO
Miguel TRAUCO
Paolo HURTADO
Christian CUEVA
Paolo GUERRERO
Jefferson FARFAN
Raul RUIDIAZ
Carlos CACEDA
Renato TAPIA
Andy POLO
Christian RAMOS
Wilder CARTAGENA
Luis ADVINCULA
Andre CARRILLO
Yoshimar YOTUN
Edison FLORES
Jose CARVALLO
Nilson LOYOLA
Pedro AQUINO
Yassine BOUNOU
Achraf HAKIMI
Hamza MENDYL
Manuel DA COSTA
Mehdi BENATIA
Ghanem SAISS
Hakim ZIYACH
Success 



In [0]:
df.head()

,id,Name,Country,Role,Age,Height(cm),International Caps,International Goals
0,275261,Pedro GALLESE,Peru,Goalkeeper,29,189.0 cm,42,0
1,202638,Alberto RODRIGUEZ,Peru,Defender,35,184.0 cm,0,0
2,306194,Aldo CORZO,Peru,Defender,30,173.0 cm,0,0
3,398457,Anderson SANTAMARIA,Peru,Defender,28,185.0 cm,0,0
4,368000,Miguel ARAUJO,Peru,Defender,25,180.0 cm,0,0
